<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_45_02_Pull_Print.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---



[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

Copyright (c) 2022, Prithwis Mukerjee All rights reserved.

This source code is licensed under the GNU GPL v3.0 -style license found in the LICENSE file in the root directory of this source tree.

#SetUp

##Installs

In [1]:
#!pip -qq install pyswisseph                                 # https://stackoverflow.com/questions/64277506/pip-install-options-unclear
!pip -qq install python-docx                #https://python-docx.readthedocs.io/en/latest/
!/usr/bin/python3 -m pip install "pymongo[srv]"
!apt install mongodb > /dev/null

     |████████████████████████████████| 5.6 MB 8.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 269 kB 7.4 MB/s 




In [2]:
#Utility functions 
from pymongo import MongoClient
import pandas as pd
import dateutil
import json
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import datetime
from datetime import timedelta
import pytz

#import matplotlib.pyplot as plt
#import math


In [3]:
#Load p21 modules from github
!wget -O p21.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21.py                  # all global variables
#!wget -O p21swe.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21swe.py            # Swiss Ephemeris functions
!wget -O p21utils.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utils.py        # large number of utility functions
!wget -O p21utilsR.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21utilsR.py 
!wget -O p21YogInfo.py -q https://raw.githubusercontent.com/prithwis/parashar21/main/utils/p21YogInfo.py
# ---------------------------

!wget -O Saraswati.png -q https://github.com/prithwis/parashar21/raw/main/images/Saraswati02.png             
!wget -O p21logo.png -q https://github.com/prithwis/parashar21/raw/main/images/p21logo-002-1.png                

In [13]:
import p21
#import p21swe
import p21utils
import p21utilsR
#import p21YogInfo
# ------------------------------------------------
# required for multiple imports of the same module
# ------------------------------------------------
import importlib
importlib.reload(p21)
#importlib.reload(p21swe)
importlib.reload(p21utils)
importlib.reload(p21utilsR)
#importlib.reload(p21YogInfo)

p21utils imported
p21utilsR imported


<module 'p21utilsR' from '/content/p21utilsR.py'>

In [5]:
from google.colab import drive
drive.mount('/content/drive')
# -------------------------------------------
# copy credentials file
!cp /content/drive/'My Drive'/Praxis/WebCredentials/Parashar21_Atlas_18Sep22.py credentials.py
# -------------------------------------------
# MongoDB credentials are stored here
#
from credentials import parashar21URI
# -------------------------------------------
mongoURI = parashar21URI                                   # for remote ATlas server
client = MongoClient(mongoURI)
client.list_database_names() # ['admin', 'local']


Mounted at /content/drive


['Parashar21', 'admin', 'local']

In [6]:
# Show existing Collections
db = client.Parashar21
pd.DataFrame(list(db.list_collections()))

,name,type,options,info,idIndex
0,khona21,collection,{},"{'readOnly': False, 'uuid': b'\xc8\x07\xffb)YI...","{'v': 2, 'key': {'_id': 1}, 'name': '_id_'}"


In [7]:
#kollection = db.khona21Test                   # Set the collection to work with
p21.kollection = db.khona21                   # Set the collection to work with
totalCharts = p21.kollection.count_documents({})
print(totalCharts)

39662


#Generate Report
for all retrieved charts

In [8]:
def genSampleReport(ChartStyle,Sample = True):
    
    if Sample:
        df4Report = df.sample(5)
    else:
        df4Report = df
    
    selectedCharts = len(df)
    printCharts = len(df4Report)
    #printStatus = str(printCharts)+' charts of total '+str(totalCharts)+' printed.'
    printStatus = str(selectedCharts)+' Charts selected from '+str(totalCharts)+' in database; '+str(printCharts)+' charts printed.'

    now = datetime.now(pytz.timezone('Asia/Kolkata'))
    fileName = 'p21Report_'+ChartStyle+now.strftime("%H%M%S")+'.doc'
    curQueryStr = json.dumps(selCond)
    # ---------------------------------------------------------
    p21utilsR.R01_CreateReportDoc(curQueryStr, printStatus)                      # Creates MS Word Doc called p21.document
    # ----------------------------------------------------------
    for chart in range(len(df4Report)):
        chartData =df4Report.iloc[chart]
        #R51_processChart(chartData)
        p21utilsR.R511_parseChartData(chartData)              # Break up database data into useful pieces
                                            # Creates two primary dicts, GLon, GRet
        for ctype in ['Rashi','Navamsa']:
        #for ctype in ['Rashi']:
            p21.ChartType = ctype
            p21utils.R11_LocateGrahaInRashi()   # Converts Graha Long to Rashi positions
                                                # Rashi Num as well as Rashi name
            if ChartStyle == 'Bengal':
                p21utilsR.R12B_drawChart_Bengal()    # Draw Chart in Bengal style
            if ChartStyle == 'South':
                p21utilsR.R12B_drawChart_South()    # Draw Chart in South style
            if ChartStyle == 'North':
                p21utilsR.R12B_drawChart_North()    # Draw Chart in North style
            #
            
                                                
        #print(p21.GrahaBhava)
        #print(p21.LordBhav)
        #print(p21.pTags)
        #print(p21.pName)
        
        p21utilsR.R30_LocateYogs()
        p21utilsR.R512_FormatPage()
        #print('----')
    # ------------------------------------
    p21.document.save(fileName)
    


#Retrieve Charts 

In [ ]:
'''
c = kollection.find().limit(5)
for crnt in c:
    print(crnt)
'''

'\nc = kollection.find().limit(5)\nfor crnt in c:\n    print(crnt)\n'

In [ ]:
#%%capture
#selCond = {}                      # All Charts 
#df = pd.DataFrame(list(kollection.find(selCond,p21.selCols)))
#genSampleReport('Bengal',Sample = False)

In [9]:
%%capture

selCond = {'GAspectedBy2.La': {'$in': ['Sa']}}                      # La aspected by Sa
df = pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))
genSampleReport('Bengal')

In [ ]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}}       # La aspected by Sa          

                    ]
}

df = pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))
genSampleReport('Bengal')

In [ ]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}} ,     # La aspected by Sa       
                        {'GConjunctsG2.Su' : {'$in': ['Me']}}       # Su conjunct Me 

                    ]
}

df = pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))
genSampleReport('North')

In [ ]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}} ,     # La aspected by Sa       
                        {'GConjunctsG2.Su' : {'$in': ['Me']}},      # Su conjunct Me 
                        {'GrahaBhava.Mo' : {'$eq': 1}}              # Mo in First House

                    ]
}

df = pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))
genSampleReport('Bengal')

In [14]:
%%capture
selCond = {"$and":
                    [
                        {'exaltG.Ju': {'$eq': True}},               # Exalted Jupiter
                        {'GAspectedBy2.La' : {'$in': ['Sa']}} ,     # La aspected by Sa       
                        {'GConjunctsG2.Su' : {'$in': ['Me']}},      # Su conjunct Me 
                        {'GrahaBhava.Mo' : {'$eq': 1}} ,            # Mo in First House
                        {'LordBhav.4' : {'$eq': 5}}                 # 4th Lord in 5th House

                    ]
}

df = pd.DataFrame(list(p21.kollection.find(selCond,p21.selCols)))
genSampleReport('Bengal',Sample = False)

In [15]:
!rm *.doc

In [16]:
!date

Thu Sep 22 05:40:47 UTC 2022


In [ ]:
#df = pd.DataFrame(list(kollection.find({'pid.tags': {"$in": ['PostGraduate']}},{'pid.name':1,"_id":0,'GLon':1,'GRet':1})))
#df